In [1]:
# 假设您使用 Python
!pip install langchain langchain-community pydantic sentence-transformers chromadb

In [3]:
!pip install langchain-google-genai
!pip install zhipuai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.1/119.1 kB 4.9 MB/s eta 0:00:00
  Attempting uninstall: pyjwt
    Found existing installation: PyJWT 2.10.1
    Uninstalling PyJWT-2.10.1:
      Successfully uninstalled PyJWT-2.10.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
mcp 1.24.0 requires pyjwt[crypto]>=2.10.1, but you have pyjwt 2.8.0 which is incompatible.


# 加载数据集并文档分割

In [4]:
from langchain_community.document_loaders import CSVLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

# 假设您的记录保存在一个名为 'baby_records.csv' 的文件中
# loader = CSVLoader("baby_records.csv", csv_args={'delimiter': ';'},encoding="utf-8")
# documents = loader.load()


## 流数据清洗及加载

In [74]:
import re

def get_actual_event_time(raw_line):
    # 1. 自动时间戳（假设在行首，如 2025-12-19 15:44:39）
    auto_time_match = re.search(r'(\d{4}-\d{2}-\d{2})\s+(\d{2}:\d{2}:\d{2})', raw_line)
    auto_date = auto_time_match.group(1) if auto_time_match else "未知日期"
    auto_time = auto_time_match.group(2) if auto_time_match else "00:00:00"

    # 2. 从内容中寻找“点/分”描述 (比如 "3点", "3点半", "15:30")
    # 匹配模式：数字 + 点/时 + (数字 + 分/半)?
    content_time_pattern = r'(\d{1,2})[点时](\d{1,2}分|半)?'
    content_match = re.search(content_time_pattern, raw_line)

    actual_time_display = auto_time # 默认值

    if content_match:
        hour = content_match.group(1)
        minute = content_match.group(2)

        # 格式化分钟
        if not minute:
            min_str = "00"
        elif "半" in minute:
            min_str = "30"
        else:
            min_str = re.sub(r'\D', '', minute).zfill(2) # 提取数字并补零

        actual_time_display = f"{hour.zfill(2)}:{min_str} (内容提及)"

    return auto_date, actual_time_display

# 测试案例：
# "2025-12-19 15:44:39, 3点睡醒了" -> 返回 ("2025-12-19", "03:00 (内容提及)")

def get_tags(text):
  # 匹配末尾或文中的 #加中文关键词 如 #辅食, #生病
    tags = re.findall(r'#(\w+)', line)

    tags = []
    if any(k in text for k in ["吃", "奶", "发糕", "南瓜", "辅食"]): tags.append("饮食")
    if any(k in text for k in ["睡", "起", "觉"]): tags.append("作息")
    if any(k in text for k in ["药", "病", "烧", "医"]): tags.append("医疗")
    if any(k in text for k in ["拉", "尿", "臭"]): tags.append("排泄")
    return " | ".join(tags) if tags else "日常"


def process_flexible_record(line):
    """
    发生时间和自动时间戳清洗，优先发生时间
    获取发生时间的时分，根据时间戳获得月份、日期
    """
    line = line.strip(" ")
    auto_date, actual_event_time = get_actual_event_time(line)
    year_month = f"{auto_date[:4]}年{int(auto_date[5:7])}月"

    # 清理掉自动时间戳，只留内容，避免干扰
    clean_content = re.sub(r'\d{4}-\d{2}-\d{2}\s+\d{2}:\d{2}:\d{2},?', '', line).strip()

    # 构造 Page Content
    # 核心：把“发生时间”作为第一特征
    enhanced_content = (
        f"发生时间: {actual_event_time}\n"
        f"日期: {auto_date}\n"
        f"月份: {year_month}\n"
        f"分类: {get_tags(clean_content)}\n" # 使用上一步建议的标签函数
        f"具体描述: {clean_content}"
    )

    return Document(
        page_content=enhanced_content,
        metadata={"date": auto_date, "month": year_month, "actual_time": actual_event_time}
    )

In [76]:
# 改成LLM打标（获取分类、事件时间）

In [75]:
line="2025-12-19 15:44:39,1点-3点午睡，3点半起来吃了半个燕麦牛奶发糕，南瓜半个"
process_flexible_record(line)

Document(metadata={'date': '2025-12-19', 'month': '2025年12月', 'actual_time': '01:00 (内容提及)'}, page_content='发生时间: 01:00 (内容提及)\n日期: 2025-12-19\n月份: 2025年12月\n分类: 饮食 | 作息\n具体描述: 1点-3点午睡，3点半起来吃了半个燕麦牛奶发糕，南瓜半个')

## excel数据加载

In [5]:
import re

# 定义一个提取日期的函数
def extract_date_info(text):
    # 匹配 yyyy-mm-dd 或 yyyy/mm/dd 格式的正则
    date_match = re.search(r'(\d{4})[-/](\d{2})[-/](\d{2})', str(text))
    if date_match:
        year, month, day = date_match.groups()
        return f"{year}-{month}-{day}", f"{year}年{int(month)}月"
    return "未知日期", "未知月份"

In [6]:
import pandas as pd
from langchain_core.documents import Document

# 您的 CSV 文件路径
file_path = './baby_records.csv'

# ！！！请根据您的实际 CSV 表头进行修改！！！
# 假设您的 CSV 列名是：'事件类型', '详情', '说明'
COLUMN_NAMES = ['事件类型', '详情', '说明']

# --- 1. 使用 Pandas 读取 CSV 文件并跳过表头 ---
try:
    df = pd.read_csv(
        file_path,
        encoding='utf8',
        header=None,     # 告诉 Pandas 文件中没有表头了
        names=COLUMN_NAMES # 手动指定列名
    )
except FileNotFoundError:
    print(f"错误：文件未找到在路径: {file_path}")
    df = None
except Exception as e:
    print(f"读取文件时发生错误: {e}")
    df = None

documents = []

if df is not None:
    # --- 2. 将 Pandas DataFrame 的每一行转换为 LangChain Document ---
    for index, row in df.iterrows():

        # A. 创建 Page Content (核心内容)
        # 目标：将数据行格式化为一个清晰、易于模型理解的描述性文本。
        content_parts = []
        # 获取列值，如果值为 NaN（缺失），则用空字符串代替
        type_record = str(row.get('事件类型', ''))
        desc = str(row.get('说明', ''))
        record_detail = str(row.get("详情", '')) if pd.notna(row.get("详情")) else ''
        raw_date, month_label = extract_date_info(record_detail)

        content_parts = [
        f"时间: {raw_date}",
        f"月份: {month_label}", # <--- 关键点：为了让 LLM 和 Embedding 识别“7月”
        f"事件类型: {type_record}",
        f"详情: {record_detail}",
        f"说明: {desc}"
        ]
        # 使用换行符或分隔符连接所有信息
        page_content = "\n".join(content_parts)

        # B. 创建 Metadata (元数据)
        # 存储所有关键信息，用于后续的元数据过滤 (Metadata Filtering)
        metadata = row.to_dict()
        metadata['source'] = file_path
        metadata['row_index'] = index
        metadata['extracted_date'] = raw_date
        metadata['month'] = month_label

        # C. 创建 LangChain Document 对象
        doc = Document(page_content=page_content, metadata=metadata)
        documents.append(doc)

    print(f"✅ 文件加载成功！共转换了 {len(documents)} 条记录。")

    # 打印一条记录验证格式
    if documents:
        print("\n--- 首条记录 Page Content 示例 ---")
        print(documents[0].page_content)
        print("\n--- 首条记录 Metadata 示例 ---")
        print(documents[0].metadata)

✅ 文件加载成功！共转换了 23 条记录。

--- 首条记录 Page Content 示例 ---
时间: 2025-04-10
月份: 2025年4月
事件类型: 生病记录
详情: 2025/04/10：未就医，扶住沙发不稳，往后磕到后脑勺。
说明: 冰敷

--- 首条记录 Metadata 示例 ---
{'事件类型': '生病记录', '详情': '2025/04/10：未就医，扶住沙发不稳，往后磕到后脑勺。', '说明': '冰敷', 'source': './baby_records.csv', 'row_index': 0, 'extracted_date': '2025-04-10', 'month': '2025年4月'}


In [8]:
print(f"载入后的变量类型为：{type(documents)}，",  f"该 Markdown 一共包含 {len(documents)} 行")

载入后的变量类型为：<class 'list'>， 该 Markdown 一共包含 23 行


In [9]:
record = documents[0]
print(f"每一个元素的类型：{type(record)}.",
    f"该文档的描述性数据：{record.metadata}",
    f"查看该文档的内容:\n{record.page_content}",
    sep="\n------\n")

每一个元素的类型：<class 'langchain_core.documents.base.Document'>.
------
该文档的描述性数据：{'事件类型': '生病记录', '详情': '2025/04/10：未就医，扶住沙发不稳，往后磕到后脑勺。', '说明': '冰敷', 'source': './baby_records.csv', 'row_index': 0, 'extracted_date': '2025-04-10', 'month': '2025年4月'}
------
查看该文档的内容:
时间: 2025-04-10
月份: 2025年4月
事件类型: 生病记录
详情: 2025/04/10：未就医，扶住沙发不稳，往后磕到后脑勺。
说明: 冰敷


In [46]:
# 2. 切分文档 (Text Splitting)
# 将长文本切分成适合检索的小片段，保留上下文信息

# csv每行都是完整的单条内容，不需要切分
# text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=10)
# split_docs = text_splitter.split_documents(documents)

In [47]:
# print(f"切分后第一个文件内容：{split_docs[0]}")

In [48]:
# print(f"切分后的文件数量：{len(split_docs)}")

In [11]:
# split_docs

In [49]:
# print(f"切分后的字符数（可以用来大致评估 token 数）：{sum([len(doc.page_content) for doc in split_docs])}")

In [ ]:
# !pip install sentence-transformers --force-reinstall --upgrade

# Embedding

In [10]:
# 3. 选择嵌入模型 (Embedding Model)
# 使用一个本地中文嵌入模型
# 注意：下载模型可能需要时间

model_name = "BAAI/bge-small-zh-v1.5"
embeddings = HuggingFaceEmbeddings(model_name=model_name)

/tmp/ipython-input-1530340379.py:6: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name=model_name)
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [11]:
query="孩子2025年9月发烧了吗"
response = embeddings.embed_query(query)

In [12]:
print(f"Embedding 的类型: {type(response)}")
# 嵌入模型的维度通常是固定的，例如 bge-small-zh-v1.5 的维度是 512
print(f"Embedding 的维度: {len(response)}")
print(f'embedding（前10）为：{response[:10]}')

Embedding 的类型: <class 'list'>
Embedding 的维度: 512
embedding（前10）为：[-0.011356472969055176, -0.03662160038948059, 0.0239725224673748, -0.04290159046649933, -0.047615159302949905, -0.019793476909399033, -0.042688239365816116, -0.008146771229803562, -0.019975846633315086, 0.0058308676816523075]


# 构建向量数据库

In [13]:
# 在构建向量库之前，抽查第 5 条记录
sample_index = min(5, len(documents)-1)
print(f"--- 准备存入向量库的第 {sample_index} 条内容 ---")
print(documents[sample_index].page_content)
print("---------------------------------------")

--- 准备存入向量库的第 5 条内容 ---
时间: 2025-07-08
月份: 2025年7月
事件类型: 生病记录
详情: 2025/07/08：呼吸科，夜咳为主，干咳
说明: 氨咖愈敏4ml每日3次、盐酸羟甲唑啉鼻用喷雾10ml每日2次、呋麻滴鼻液➕氯化钠（必要时）
---------------------------------------


In [16]:
# !rm -rf ./chroma_db#旧数据库干扰
# import shutil
# import os

# persist_directory = './chroma_db'

# # 如果文件夹存在，先删除它
# if os.path.exists(persist_directory):
#     try:
#         shutil.rmtree(persist_directory)
#         print(f"✅ 已成功清理旧的数据库目录: {persist_directory}")
#     except Exception as e:
#         print(f"❌ 无法删除文件夹，请手动删除或检查权限: {e}")

In [15]:
# 4. 创建向量数据库 (Vector Store)
# 使用 ChromaDB 在本地存储向量
# persist_directory 用于保存数据，下次可以直接加载
persist_directory = './chroma_db'
#创建向量数据库
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings,
    persist_directory=persist_directory
)

#从缓存数据库中加载向量数据库
# vectorstore = Chroma(
#     # documents=split_docs,
#     embedding_function=embeddings,
#     persist_directory=persist_directory
# )

vectorstore.persist()
print("向量数据库构建完成。")

向量数据库构建完成。


/tmp/ipython-input-1613844179.py:19: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectorstore.persist()


In [17]:
print(f"向量库中存储的数量：{vectorstore._collection.count()}")

向量库中存储的数量：23


In [55]:
import re
from langchain_core.runnables import RunnableLambda

def smart_retrieval_logic(query):
    # 1. 尝试从用户问题中提取月份（例如 "7月"）
    month_match = re.search(r'(\d+)月', query)

    if month_match:
        target_month = f"2025年{month_match.group(1)}月"
        print(f"🔍 触发动态过滤：仅检索 {target_month} 的数据")

        # 使用带有 filter 的搜索
        # 注意：这里的 'month' 必须是你存入 metadata 的 key
        return vectorstore.similarity_search(
            query,
            k=10,
            filter={"month": target_month}
        )
    else:
        # 2. 如果没匹配到月份，执行常规检索
        print("🌐 执行常规语义检索")
        return vectorstore.similarity_search(query, k=5)

# 将函数包装成一个 LangChain 可调用的组件
dynamic_retriever = RunnableLambda(smart_retrieval_logic)

In [56]:
# 使用方式
sim_docs = smart_retrieval_logic("2025年9月发烧了吗")

🔍 触发动态过滤：仅检索 2025年9月 的数据


In [48]:
question = "2025年9月发烧了吗"
sim_docs = vectorstore.similarity_search(question,k=3)

In [19]:
print(f"检索到的内容数：{len(sim_docs)}")

检索到的内容数：3


In [53]:
for i, sim_doc in enumerate(sim_docs):
    print(f"检索到的第{i}个内容: \n{sim_doc.page_content}", end="\n--------------\n")

检索到的第0个内容: 
时间: 2025-09-07
月份: 2025年9月
事件类型: 生病记录
详情: 2025/09/07：内科，早上发烧，体温38度，流鼻涕不咳嗽，中性粒细胞59.1高，淋巴细胞28.4低，嗜酸粒细胞560
说明: 头孢30mg每日2次、小美林布洛芬38.5度以上使用
--------------
检索到的第1个内容: 
时间: 2025-09-26
月份: 2025年9月
事件类型: 生病记录
详情: 2025/09/26：中医科，反复咳嗽3个月，夜咳，咳白痰
说明: 小儿消积止咳口服液5ml每日3次
--------------
检索到的第2个内容: 
时间: 2025-09-12
月份: 2025年9月
事件类型: 生病记录
详情: 2025/09/12：皮肤科，慢性荨麻疹半年，急性过敏源牛奶、虾、狗毛，慢性过敏源鸡蛋、牛奶
说明: 盐酸西替每日5滴睡前1次口服3个月
--------------


In [59]:
mmr_docs = vectorstore.max_marginal_relevance_search(question,k=3)
for i, sim_doc in enumerate(mmr_docs):
    print(f"MMR 检索到的第{i}个内容: \n{sim_doc.page_content[:200]}", end="\n--------------\n")

MMR 检索到的第0个内容: 
详情: 2025/12/07：未就医，早上10点发烧38度，下午4点发烧38.5度
--------------
MMR 检索到的第1个内容: 
时间: 2025-12-08
月份: 2025年12月
事件类型: 生病记录
详情: 2025/12/08：未就医，下午4点发烧37.7度
说明: 小儿柴桂口服液每日4次
--------------
MMR 检索到的第2个内容: 
时间: 2025-09-12
月份: 2025年9月
事件类型: 生病记录
--------------


# LLM客户端

In [77]:
!touch .env
!echo "GEMINI_API_KEY=your-gemini" > .env
!echo "ZHIPUAI_API_KEY=your-zhipu" >> .env

In [22]:
from dotenv import find_dotenv, load_dotenv
import os
_ = load_dotenv(find_dotenv())
# 获取环境变量 API_KEY
zhipu_api_key = os.environ["ZHIPUAI_API_KEY"]
gemini_api_key = os.environ["GEMINI_API_KEY"]

In [24]:
from google import genai
from google.genai.errors import APIError
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_core.output_parsers import StrOutputParser

gemini_llm = ChatGoogleGenerativeAI(
    model="gemini-2.5-flash",
    temperature=0  # 如果需要更事实、更准确的回答，可以设置较低的温度
)

In [25]:
from langchain_community.chat_models import ChatZhipuAI

zhipu_llm = ChatZhipuAI(
    model="glm-4",  # 指定您想使用的模型，例如 GLM-4
    temperature=0.1 # RAG 任务通常倾向于低温度以提高事实准确性
)

In [26]:
# 请仔细检查上下文中的‘日期’字段。如果用户询问特定月份，请扫描所有提供的记录后再回答。
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
# 2. 定义文档格式化函数
# 这个函数的作用就是替代 create_stuff_documents_chain 中处理文档的步骤
def format_docs(docs):
    # 将检索到的所有文档内容连接成一个大字符串，作为 {context} 变量
    return "\n\n".join(doc.page_content for doc in docs)


template = """
你是一个专业的育儿助手，请基于以下提供的孩子成长记录片段，
用清晰、友好的中文回答用户的问题。
如果记录中没有足够的信息，请礼貌地告知用户。

检索到的成长记录片段（上下文）：
{context}

用户问题: {input}
"""
prompt = ChatPromptTemplate.from_template(template)

In [57]:
# retriever = vectorstore.as_retriever(search_kwargs={"k": 3})
docs = dynamic_retriever.invoke(question)
print(f"检索到的内容数：{len(docs)}")

🔍 触发动态过滤：仅检索 2025年9月 的数据
检索到的内容数：3


In [58]:
for i, doc in enumerate(docs):
    print(f"检索到的第{i}个内容: \n {doc.page_content}", end="\n-----------------------------------------------------\n")

检索到的第0个内容: 
 时间: 2025-09-07
月份: 2025年9月
事件类型: 生病记录
详情: 2025/09/07：内科，早上发烧，体温38度，流鼻涕不咳嗽，中性粒细胞59.1高，淋巴细胞28.4低，嗜酸粒细胞560
说明: 头孢30mg每日2次、小美林布洛芬38.5度以上使用
-----------------------------------------------------
检索到的第1个内容: 
 时间: 2025-09-26
月份: 2025年9月
事件类型: 生病记录
详情: 2025/09/26：中医科，反复咳嗽3个月，夜咳，咳白痰
说明: 小儿消积止咳口服液5ml每日3次
-----------------------------------------------------
检索到的第2个内容: 
 时间: 2025-09-12
月份: 2025年9月
事件类型: 生病记录
详情: 2025/09/12：皮肤科，慢性荨麻疹半年，急性过敏源牛奶、虾、狗毛，慢性过敏源鸡蛋、牛奶
说明: 盐酸西替每日5滴睡前1次口服3个月
-----------------------------------------------------


In [59]:
query = "2025年7月孩子情况"
docs = dynamic_retriever.invoke(query)
for doc in docs:
    # print(doc.page_content)
    # print(doc.metadata)
    print(doc.metadata['extracted_date']) # 看看检索出来的到底是不是 7 月的？

🔍 触发动态过滤：仅检索 2025年7月 的数据
2025-07-02
2025-07-18
2025-07-02
2025-07-08


# qa_chain

In [60]:
gemini_rag_chain = (
    # Step 1: 准备输入字典 (将用户输入复制到 'question' 键)
    {"context": dynamic_retriever | format_docs, "input": RunnablePassthrough()}
    # Step 2: 将输入字典传递给 Prompt
    | prompt
    # Step 3: 将填充好的 Prompt 传递给 LLM
    | gemini_llm
    # Step 4: 将 LLM 输出转换为字符串
    | StrOutputParser()
)

In [61]:
zhipu_rag_chain = (
    # Step 1: 准备输入字典 (将用户输入复制到 'question' 键)
    {"context": dynamic_retriever | format_docs, "input": RunnablePassthrough()}
    # Step 2: 将输入字典传递给 Prompt
    | prompt
    # Step 3: 将填充好的 Prompt 传递给 LLM
    | zhipu_llm
    # Step 4: 将 LLM 输出转换为字符串
    | StrOutputParser()
)

In [65]:
question_1="2025年12月去了几次医院"
# question_1="最近一次发烧是什么时候"
result = gemini_rag_chain.invoke(question_1)
print("大模型+知识库后回答 question_1 的结果：")
print(result)

🔍 触发动态过滤：仅检索 2025年12月 的数据
大模型+知识库后回答 question_1 的结果：
您好！根据提供的成长记录，在2025年12月，孩子去了医院 **2次**：

1.  **2025年12月04日**：去了中医科，处理积食问题。
2.  **2025年12月05日**：去了骨科，处理手腕脱臼。

其他生病记录（如12月07日的发烧、12月08日的发烧、12月09日的轻微感冒）都注明了“未就医”或没有提及去医院。


In [64]:
question_1="2025年8月去了几次医院"
result = zhipu_rag_chain.invoke(question_1)
print("大模型+知识库后回答 question_1 的结果：")
print(result)

🔍 触发动态过滤：仅检索 2025年8月 的数据
大模型+知识库后回答 question_1 的结果：
孩子在2025年8月份一共去了三次医院，具体记录如下：

1. 2025年8月3日的呼吸科就诊记录。
2. 2025年8月8日的中医科就诊记录。
3. 2025年8月11日的中医科就诊记录。

请注意，这些记录都是关于生病就诊的情况。希望孩子在这些治疗后能够早日康复。如果您需要更多关于孩子健康状况的信息，请随时告知。


若RAG效果不好，由于几个原因导致的？
1. 提问日期的时候，记录里有7月的，但回答没有？
*   load数据的是，每条时间记录加载为一行
*   修改系统prompt
*   load数据的时候，把7月提取出来，如果检索里没有，是embedding问题。如果检索里有，就是llm的问题
    * 把日期单独提取出来，放到metadate里，可以找到检索到相关日期
    * 利用metadata元数据过滤时间，时间检索的时候，7月和9月是相似emb，需要元数据过滤

    * 只有记录类型，没有记录详情？（存入向量数据库之前记录是完整的，旧向量数据库未清空，重启会话）

    


数据记录更方便，表格记录改成流数据记录，每日通过飞书机器人发送消息到云，云上保存每条记录（术语？）
+ 清洗数据：发生时间和记录时间不一致
+ 记录的时候，不同记录加\#分类，方便洗数据